# ETL using BigQuery

## Importing Libraries

In [1]:
from google.cloud import bigquery
import pandas as pd
from datetime import datetime
import os
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery_storage
import google.cloud.bigquery.magics
import pandas_gbq

## Loading Credentials file

In [2]:
credentials1 = service_account.Credentials.from_service_account_file("eighth-alchemy-382219-a8d58d2f86d6.json")

# Loading Dataset

In [3]:
df = pd.read_excel('HR Dataset - DWH.xlsx', sheet_name='HR Dataset')

In [4]:
df.head()

,Employee_Name,Employee_Number,State,Zip,DOB,Age,Sex,MaritalDesc,CitizenDesc,Hispanic_Latino,...,Date_of_Hire,Date_of_Termination,Reason_For_Term,Employment_Status,Department,Position,Pay_Rate,Manager_Name,Employee_Source,Performance_Score
0,"Brown, Mia",1103024456,MA,1450,1985-11-24,32,Female,Married,US Citizen,No,...,2008-10-27,NaT,N/A - still employed,Active,Admin Offices,Accountant I,28.50,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets
1,"LaRotonda, William",1106026572,MA,1460,1984-04-26,33,Male,Divorced,US Citizen,No,...,2014-01-06,NaT,N/A - still employed,Active,Admin Offices,Accountant I,23.00,Brandon R. LeBlanc,Website Banner Ads,Fully Meets
2,"Steans, Tyrone",1302053333,MA,2703,1986-09-01,31,Male,Single,US Citizen,No,...,2014-09-29,NaT,N/A - still employed,Active,Admin Offices,Accountant I,29.00,Brandon R. LeBlanc,Internet Search,Fully Meets
3,"Howard, Estelle",1211050782,MA,2170,1985-09-16,32,Female,Married,US Citizen,No,...,2015-02-16,2015-04-15,N/A - still employed,Active,Admin Offices,Administrative Assistant,21.50,Brandon R. LeBlanc,Pay Per Click - Google,N/A- too early to review
4,"Singh, Nan",1307059817,MA,2330,1988-05-19,29,Female,Single,US Citizen,No,...,2015-05-01,NaT,N/A - still employed,Active,Admin Offices,Administrative Assistant,16.56,Brandon R. LeBlanc,Website Banner Ads,N/A- too early to review


In [5]:
df['Date_of_Termination'] = df['Date_of_Termination'].fillna(datetime(9999, 12, 31))


In [6]:
df['Reason_For_Term'] = df['Reason_For_Term'].str.replace('N/A - ', '')

In [7]:
df['Performance_Score'] = df['Performance_Score'].str.replace('N/A- ', '')

In [8]:
df[['First_Name', 'Last_Name']] = df['Employee_Name'].str.split(',', expand=True)

# drop the original Employee Name column
df.drop('Employee_Name', axis=1, inplace=True)

# reorder the columns to have the new columns first
df = df[['First_Name', 'Last_Name'] + list(df.columns[:-2])]

In [9]:
df1 = pd.read_excel('HR Dataset - DWH.xlsx', sheet_name='HR_data')

In [10]:
df1.head()

,Employee_Name,Employee_Number,MarriedID,MaritalStatusID,GenderID,EmpStatus_ID,DeptID,Perf_ScoreID,Age,Pay_Rate,...,Date_of_Hire,Days_Employed,Date_of_Termination,Reason_For_Term,Employment_Status,Department,Position,Manager_Name,Employee_Source,Performance_Score
0,"Brown, Mia",1103024456,1,1,0,1,1,3,30,28.50,...,2008-10-27,3317,NaT,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets
1,"LaRotonda, William",1106026572,0,2,1,1,1,3,34,23.00,...,2014-01-06,1420,NaT,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Website Banner Ads,Fully Meets
2,"Steans, Tyrone",1302053333,0,0,1,1,1,3,31,29.00,...,2014-09-29,1154,NaT,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Internet Search,Fully Meets
3,"Howard, Estelle",1211050782,1,1,0,1,1,9,32,21.50,...,2015-02-16,58,2015-04-15,N/A - still employed,Active,Admin Offices,Administrative Assistant,Brandon R. LeBlanc,Pay Per Click - Google,N/A- too early to review
4,"Singh, Nan",1307059817,0,0,0,1,1,9,30,16.56,...,2015-05-01,940,NaT,N/A - still employed,Active,Admin Offices,Administrative Assistant,Brandon R. LeBlanc,Website Banner Ads,N/A- too early to review


In [11]:
df1['Reason_For_Term'] = df1['Reason_For_Term'].str.replace('N/A - ', '')
df1['Performance_Score'] = df1['Performance_Score'].str.replace('N/A- ', '')


In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Employee_Name        310 non-null    object        
 1   Employee_Number      310 non-null    int64         
 2   MarriedID            310 non-null    int64         
 3   MaritalStatusID      310 non-null    int64         
 4   GenderID             310 non-null    int64         
 5   EmpStatus_ID         310 non-null    int64         
 6   DeptID               310 non-null    int64         
 7   Perf_ScoreID         310 non-null    int64         
 8   Age                  310 non-null    int64         
 9   Pay_Rate             310 non-null    float64       
 10  State                310 non-null    object        
 11  Zip                  310 non-null    int64         
 12  DOB                  310 non-null    datetime64[ns]
 13  Sex                  310 non-null  

In [13]:
merged_df = pd.merge(df, df1, on='Employee_Number', how='inner')

In [14]:
merged_df.head()

,First_Name,Last_Name,Employee_Number,State_x,Zip_x,DOB_x,Age_x,Sex_x,MaritalDesc_x,CitizenDesc_x,...,Date_of_Hire_y,Days_Employed,Date_of_Termination_y,Reason_For_Term_y,Employment_Status_y,Department_y,Position_y,Manager_Name_y,Employee_Source_y,Performance_Score_y
0,Brown,Mia,1103024456,MA,1450,1985-11-24,32,Female,Married,US Citizen,...,2008-10-27,3317,NaT,still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets
1,LaRotonda,William,1106026572,MA,1460,1984-04-26,33,Male,Divorced,US Citizen,...,2014-01-06,1420,NaT,still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Website Banner Ads,Fully Meets
2,Steans,Tyrone,1302053333,MA,2703,1986-09-01,31,Male,Single,US Citizen,...,2014-09-29,1154,NaT,still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Internet Search,Fully Meets
3,Howard,Estelle,1211050782,MA,2170,1985-09-16,32,Female,Married,US Citizen,...,2015-02-16,58,2015-04-15,still employed,Active,Admin Offices,Administrative Assistant,Brandon R. LeBlanc,Pay Per Click - Google,too early to review
4,Singh,Nan,1307059817,MA,2330,1988-05-19,29,Female,Single,US Citizen,...,2015-05-01,940,NaT,still employed,Active,Admin Offices,Administrative Assistant,Brandon R. LeBlanc,Website Banner Ads,too early to review


In [15]:
merged_df.columns

Index(['First_Name', 'Last_Name', 'Employee_Number', 'State_x', 'Zip_x',
       'DOB_x', 'Age_x', 'Sex_x', 'MaritalDesc_x', 'CitizenDesc_x',
       'Hispanic_Latino_x', 'RaceDesc_x', 'Date_of_Hire_x',
       'Date_of_Termination_x', 'Reason_For_Term_x', 'Employment_Status_x',
       'Department_x', 'Position_x', 'Pay_Rate_x', 'Manager_Name_x',
       'Employee_Source_x', 'Performance_Score_x', 'Employee_Name',
       'MarriedID', 'MaritalStatusID', 'GenderID', 'EmpStatus_ID', 'DeptID',
       'Perf_ScoreID', 'Age_y', 'Pay_Rate_y', 'State_y', 'Zip_y', 'DOB_y',
       'Sex_y', 'MaritalDesc_y', 'CitizenDesc_y', 'Hispanic_Latino_y',
       'RaceDesc_y', 'Date_of_Hire_y', 'Days_Employed',
       'Date_of_Termination_y', 'Reason_For_Term_y', 'Employment_Status_y',
       'Department_y', 'Position_y', 'Manager_Name_y', 'Employee_Source_y',
       'Performance_Score_y'],
      dtype='object')

In [16]:
repeating_columns = ['Sex_y','Age_y','Pay_Rate_y','State_y','Zip_y','DOB_y','MaritalDesc_y','CitizenDesc_y', 'Hispanic_Latino_y',
       'RaceDesc_y', 'Date_of_Hire_y','Date_of_Termination_y', 'Reason_For_Term_y', 'Employment_Status_y',
       'Department_y', 'Position_y', 'Manager_Name_y', 'Employee_Source_y',
       'Performance_Score_y']

In [17]:
merged_df = merged_df.drop(repeating_columns, axis=1)


In [18]:
merged_df.columns

Index(['First_Name', 'Last_Name', 'Employee_Number', 'State_x', 'Zip_x',
       'DOB_x', 'Age_x', 'Sex_x', 'MaritalDesc_x', 'CitizenDesc_x',
       'Hispanic_Latino_x', 'RaceDesc_x', 'Date_of_Hire_x',
       'Date_of_Termination_x', 'Reason_For_Term_x', 'Employment_Status_x',
       'Department_x', 'Position_x', 'Pay_Rate_x', 'Manager_Name_x',
       'Employee_Source_x', 'Performance_Score_x', 'Employee_Name',
       'MarriedID', 'MaritalStatusID', 'GenderID', 'EmpStatus_ID', 'DeptID',
       'Perf_ScoreID', 'Days_Employed'],
      dtype='object')

In [19]:
merged_df.columns = [col.replace('_x', '') for col in merged_df.columns]


In [20]:
merged_df = merged_df.drop('Employee_Name',axis=1)

In [21]:
merged_df

,First_Name,Last_Name,Employee_Number,State,Zip,DOB,Age,Sex,MaritalDesc,CitizenDesc,...,Manager_Name,Employee_Source,Performance_Score,MarriedID,MaritalStatusID,GenderID,EmpStatus_ID,DeptID,Perf_ScoreID,Days_Employed
0,Brown,Mia,1103024456,MA,1450,1985-11-24,32,Female,Married,US Citizen,...,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,1,1,0,1,1,3,3317
1,LaRotonda,William,1106026572,MA,1460,1984-04-26,33,Male,Divorced,US Citizen,...,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,0,2,1,1,1,3,1420
2,Steans,Tyrone,1302053333,MA,2703,1986-09-01,31,Male,Single,US Citizen,...,Brandon R. LeBlanc,Internet Search,Fully Meets,0,0,1,1,1,3,1154
3,Howard,Estelle,1211050782,MA,2170,1985-09-16,32,Female,Married,US Citizen,...,Brandon R. LeBlanc,Pay Per Click - Google,too early to review,1,1,0,1,1,9,58
4,Singh,Nan,1307059817,MA,2330,1988-05-19,29,Female,Single,US Citizen,...,Brandon R. LeBlanc,Website Banner Ads,too early to review,0,0,0,1,1,9,940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,Patronick,Luke,1112030979,MA,1844,1979-02-20,38,Male,Single,US Citizen,...,Alex Sweetwater,Diversity Job Fair,Exceeds,0,0,1,5,4,4,1400
299,Saada,Adell,1012023185,MA,2132,1986-07-24,31,Female,Married,US Citizen,...,Alex Sweetwater,Pay Per Click - Google,Fully Meets,1,1,0,1,4,3,1847
300,Szabo,Andrew,1201031324,MA,2140,1983-05-06,34,Male,Single,US Citizen,...,Alex Sweetwater,MBTA ads,Exceptional,0,0,1,1,4,5,1238
301,True,Edward,1102024057,MA,2451,1983-06-14,34,Male,Single,Non-Citizen,...,Alex Sweetwater,Diversity Job Fair,Fully Meets,0,0,1,5,4,3,421


In [22]:
salary_data = pd.read_excel('HR Dataset - DWH.xlsx', sheet_name='Salary_data')

In [23]:
salary_data

,Position,Min,Mid,Max,Unnamed: 4,Unnamed: 5
0,Administrative Assistant,30000,40000,50000,NaN,NaN
1,Sr. Administrative Assistant,35000,45000,55000,NaN,NaN
2,Accountant I,42274,51425,62299,NaN,NaN
3,Accountant II,50490,62158,74658,NaN,NaN
4,Sr. Accountant,63264,76988,92454,NaN,NaN
5,Network Engineer,50845,66850,88279,NaN,NaN
6,Sr. Network Engineer,79428,99458,120451,NaN,NaN
7,Database Administrator,50569,68306,93312,NaN,NaN
8,Sr. DBA,92863,116007,139170,NaN,NaN
9,Production Technician I,30000,40000,50000,NaN,2.0


In [24]:
salary = salary_data.drop(['Unnamed: 4', 'Unnamed: 5'], axis=1)


In [25]:
final_data = pd.merge(merged_df, salary, on='Position', how='inner')

In [26]:
final_data.head()

,First_Name,Last_Name,Employee_Number,State,Zip,DOB,Age,Sex,MaritalDesc,CitizenDesc,...,MarriedID,MaritalStatusID,GenderID,EmpStatus_ID,DeptID,Perf_ScoreID,Days_Employed,Min,Mid,Max
0,Brown,Mia,1103024456,MA,1450,1985-11-24,32,Female,Married,US Citizen,...,1,1,0,1,1,3,3317,42274,51425,62299
1,LaRotonda,William,1106026572,MA,1460,1984-04-26,33,Male,Divorced,US Citizen,...,0,2,1,1,1,3,1420,42274,51425,62299
2,Steans,Tyrone,1302053333,MA,2703,1986-09-01,31,Male,Single,US Citizen,...,0,0,1,1,1,3,1154,42274,51425,62299
3,Howard,Estelle,1211050782,MA,2170,1985-09-16,32,Female,Married,US Citizen,...,1,1,0,1,1,9,58,30000,40000,50000
4,Singh,Nan,1307059817,MA,2330,1988-05-19,29,Female,Single,US Citizen,...,0,0,0,1,1,9,940,30000,40000,50000


In [27]:
final_data.columns

Index(['First_Name', 'Last_Name', 'Employee_Number', 'State', 'Zip', 'DOB',
       'Age', 'Sex', 'MaritalDesc', 'CitizenDesc', 'Hispanic_Latino',
       'RaceDesc', 'Date_of_Hire', 'Date_of_Termination', 'Reason_For_Term',
       'Employment_Status', 'Department', 'Position', 'Pay_Rate',
       'Manager_Name', 'Employee_Source', 'Performance_Score', 'MarriedID',
       'MaritalStatusID', 'GenderID', 'EmpStatus_ID', 'DeptID', 'Perf_ScoreID',
       'Days_Employed', 'Min', 'Mid', 'Max'],
      dtype='object')

## Creating Schema for BigQuery Table

In [28]:
table_id = 'myworkingdataset.ETL_HR_Dataset'

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField('Employee_Number', 'INTEGER', mode='REQUIRED'),
        bigquery.SchemaField('First_Name', 'STRING'),
        bigquery.SchemaField('Last_Name', 'STRING'),
        bigquery.SchemaField('State', 'STRING'),
        bigquery.SchemaField('Zip', 'INTEGER'),
        bigquery.SchemaField('DOB', 'DATE'),
        bigquery.SchemaField('Age', 'INTEGER'),
        bigquery.SchemaField('Sex', 'STRING'),
        bigquery.SchemaField('MaritalDesc', 'STRING'),
        bigquery.SchemaField('CitizenDesc', 'STRING'),
        bigquery.SchemaField('Hispanic_Latino', 'STRING'),
        bigquery.SchemaField('RaceDesc', 'STRING'),
        bigquery.SchemaField('Date_of_Hire', 'DATE'),
        bigquery.SchemaField('Date_of_Termination', 'DATE'),
        bigquery.SchemaField('Reason_For_Term', 'STRING'),
        bigquery.SchemaField('Employment_Status', 'STRING'),
        bigquery.SchemaField('Department', 'STRING'),
        bigquery.SchemaField('Position', 'STRING'),
        bigquery.SchemaField('Pay_Rate', 'FLOAT'),
        bigquery.SchemaField('Manager_Name', 'STRING'),
        bigquery.SchemaField('Employee_Source', 'STRING'),
        bigquery.SchemaField('Performance_Score', 'STRING'),
        bigquery.SchemaField('MarriedID', 'INTEGER'),
        bigquery.SchemaField('MaritalStatusID', 'INTEGER'),
        bigquery.SchemaField('GenderID', 'INTEGER'),
        bigquery.SchemaField('EmpStatus_ID', 'INTEGER'),
        bigquery.SchemaField('DeptID', 'INTEGER'),
        bigquery.SchemaField('Perf_ScoreID', 'INTEGER'),
        bigquery.SchemaField('Days_Employed', 'INTEGER'),
        bigquery.SchemaField('Min', 'INTEGER'),
        bigquery.SchemaField('Mid', 'INTEGER'),
        bigquery.SchemaField('Max', 'INTEGER')
    ],
    write_disposition='WRITE_TRUNCATE' # Options: WRITE_APPEND, WRITE_TRUNCATE, WRITE_EMPTY
)


## Loading the Dataframe into BigQuery Table

In [29]:
project_id1 = 'eighth-alchemy-382219'
client = bigquery.Client(project=project_id1, credentials=credentials1)
job = client.load_table_from_dataframe(final_data, table_id, job_config=job_config)
job.result()

LoadJob<project=eighth-alchemy-382219, location=US, id=3e36219f-9c71-4244-9bb3-6c85b12d39eb>

## Querying Data from BigQuery Table

In [31]:
query = "select *  from myworkingdataset.ETL_HR_Dataset"
data = client.query(query, project=project_id1).to_dataframe()

In [32]:
data.head()

,First_Name,Last_Name,Employee_Number,State,Zip,DOB,Age,Sex,MaritalDesc,CitizenDesc,...,MarriedID,MaritalStatusID,GenderID,EmpStatus_ID,DeptID,Perf_ScoreID,Days_Employed,Min,Mid,Max
0,Steans,Tyrone,1302053333,MA,2703,1986-09-01,31,Male,Single,US Citizen,...,0,0,1,1,1,3,1154,42274,51425,62299
1,Singh,Nan,1307059817,MA,2330,1988-05-19,29,Female,Single,US Citizen,...,0,0,0,1,1,9,940,30000,40000,50000
2,Horton,Jayne,1105025718,MA,2493,1984-02-21,33,Female,Single,US Citizen,...,0,0,0,1,3,9,972,50569,68306,93312
3,Zhou,Julia,1110029732,MA,2148,1979-02-24,38,Female,Single,US Citizen,...,0,0,0,1,3,0,972,50569,68306,93312
4,Merlos,Carlos,1012023013,MA,2138,1987-06-18,30,Male,Single,US Citizen,...,0,0,1,1,3,9,972,50845,66850,88279


In [33]:
query = "SELECT * FROM myworkingdataset.ETL_HR_Dataset where Sex = 'Male' and MaritalDesc = 'Single'"
data = client.query(query, project=project_id1).to_dataframe()

In [34]:
data.head(40)

,First_Name,Last_Name,Employee_Number,State,Zip,DOB,Age,Sex,MaritalDesc,CitizenDesc,...,MarriedID,MaritalStatusID,GenderID,EmpStatus_ID,DeptID,Perf_ScoreID,Days_Employed,Min,Mid,Max
0,Steans,Tyrone,1302053333,MA,2703,1986-09-01,31,Male,Single,US Citizen,...,0,0,1,1,1,3,1154,42274,51425,62299
1,Merlos,Carlos,1012023013,MA,2138,1987-06-18,30,Male,Single,US Citizen,...,0,0,1,1,3,9,972,50845,66850,88279
2,South,Joe,1308060959,CT,6278,1965-09-09,52,Male,Single,US Citizen,...,0,0,1,1,3,0,1112,79428,99458,120451
3,Adinolfi,Wilson K,1409070522,MA,1960,1983-07-10,34,Male,Single,US Citizen,...,0,0,1,1,5,4,2336,30000,40000,50000
4,Barone,Francesco A,1101023679,MA,1810,1983-07-20,34,Male,Single,US Citizen,...,0,0,1,1,5,3,2106,30000,40000,50000
5,Bernstein,Sean,1109029366,MA,2072,1970-12-22,47,Male,Single,US Citizen,...,0,0,1,1,5,3,2064,30000,40000,50000
6,Chang,Donovan E,1111030129,MA,1845,1983-08-24,34,Male,Single,US Citizen,...,0,0,1,1,5,3,1602,30000,40000,50000
7,Cockel,James,1599991009,MA,2452,1977-09-08,40,Male,Single,US Citizen,...,0,0,1,1,5,3,1602,30000,40000,50000
8,Dickinson,Geoff,706006285,MA,2180,1982-11-15,35,Male,Single,US Citizen,...,0,0,1,1,5,3,1294,30000,40000,50000
9,Engdahl,Jean,1007020403,MA,2026,1974-05-31,43,Male,Single,US Citizen,...,0,0,1,1,5,3,1112,30000,40000,50000


In [35]:
query = "SELECT * FROM myworkingdataset.ETL_HR_Dataset where Sex = 'Male' and MaritalDesc = 'Single' and Age >30"
data = client.query(query, project=project_id1).to_dataframe()

In [36]:
data.head(50)

,First_Name,Last_Name,Employee_Number,State,Zip,DOB,Age,Sex,MaritalDesc,CitizenDesc,...,MarriedID,MaritalStatusID,GenderID,EmpStatus_ID,DeptID,Perf_ScoreID,Days_Employed,Min,Mid,Max
0,Steans,Tyrone,1302053333,MA,2703,1986-09-01,31,Male,Single,US Citizen,...,0,0,1,1,1,3,1154,42274,51425,62299
1,South,Joe,1308060959,CT,6278,1965-09-09,52,Male,Single,US Citizen,...,0,0,1,1,3,0,1112,79428,99458,120451
2,Adinolfi,Wilson K,1409070522,MA,1960,1983-07-10,34,Male,Single,US Citizen,...,0,0,1,1,5,4,2336,30000,40000,50000
3,Barone,Francesco A,1101023679,MA,1810,1983-07-20,34,Male,Single,US Citizen,...,0,0,1,1,5,3,2106,30000,40000,50000
4,Bernstein,Sean,1109029366,MA,2072,1970-12-22,47,Male,Single,US Citizen,...,0,0,1,1,5,3,2064,30000,40000,50000
5,Chang,Donovan E,1111030129,MA,1845,1983-08-24,34,Male,Single,US Citizen,...,0,0,1,1,5,3,1602,30000,40000,50000
6,Cockel,James,1599991009,MA,2452,1977-09-08,40,Male,Single,US Citizen,...,0,0,1,1,5,3,1602,30000,40000,50000
7,Dickinson,Geoff,706006285,MA,2180,1982-11-15,35,Male,Single,US Citizen,...,0,0,1,1,5,3,1294,30000,40000,50000
8,Engdahl,Jean,1007020403,MA,2026,1974-05-31,43,Male,Single,US Citizen,...,0,0,1,1,5,3,1112,30000,40000,50000
9,Garcia,Raul,1309061015,MA,1905,1985-09-15,32,Male,Single,US Citizen,...,0,0,1,1,5,9,972,30000,40000,50000
